<a href="https://colab.research.google.com/github/aflores/samples/blob/master/no-shows-sklearn-and-tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [2]:
data_path = 'https://hatchboard.com/no-shows.txt'
df = pd.read_csv(data_path) 
df.dtypes

PatientId         float64
AppointmentID       int64
Gender             object
ScheduledDay       object
AppointmentDay     object
Age                 int64
Neighbourhood      object
Scholarship         int64
Hipertension        int64
Diabetes            int64
Alcoholism          int64
Handcap             int64
SMS_received        int64
No-show            object
dtype: object

In [3]:
# from the description, looks like most data is categorical so let's see what we have in the objects
for col in df.columns:
    if (df[col].dtype == 'object'):
        print(col, df[col].unique())  

Gender ['F' 'M']
ScheduledDay ['2016-04-29T18:38:08Z' '2016-04-29T16:08:27Z' '2016-04-29T16:19:04Z' ...
 '2016-04-27T16:03:52Z' '2016-04-27T15:09:23Z' '2016-04-27T13:30:56Z']
AppointmentDay ['2016-04-29T00:00:00Z' '2016-05-03T00:00:00Z' '2016-05-10T00:00:00Z'
 '2016-05-17T00:00:00Z' '2016-05-24T00:00:00Z' '2016-05-31T00:00:00Z'
 '2016-05-02T00:00:00Z' '2016-05-30T00:00:00Z' '2016-05-16T00:00:00Z'
 '2016-05-04T00:00:00Z' '2016-05-19T00:00:00Z' '2016-05-12T00:00:00Z'
 '2016-05-06T00:00:00Z' '2016-05-20T00:00:00Z' '2016-05-05T00:00:00Z'
 '2016-05-13T00:00:00Z' '2016-05-09T00:00:00Z' '2016-05-25T00:00:00Z'
 '2016-05-11T00:00:00Z' '2016-05-18T00:00:00Z' '2016-05-14T00:00:00Z'
 '2016-06-02T00:00:00Z' '2016-06-03T00:00:00Z' '2016-06-06T00:00:00Z'
 '2016-06-07T00:00:00Z' '2016-06-01T00:00:00Z' '2016-06-08T00:00:00Z']
Neighbourhood ['JARDIM DA PENHA' 'MATA DA PRAIA' 'PONTAL DE CAMBURI' 'REPÚBLICA'
 'GOIABEIRAS' 'ANDORINHAS' 'CONQUISTA' 'NOVA PALESTINA' 'DA PENHA'
 'TABUAZEIRO' 'BENTO FERREIRA' 

In [4]:
# encode Neighbourhood
le = LabelEncoder()

t = df['Neighbourhood'].unique().astype(str)
df['Neighbourhood'] = le.fit_transform(df['Neighbourhood'])
le.classes_
le.transform(t)

t = df['Gender'].unique().astype(str)
df['Gender'] = le.fit_transform(df['Gender'])
le.classes_
le.transform(t)

t = df['No-show'].unique().astype(str)
df['No-show'] = le.fit_transform(df['No-show'])
le.classes_
le.transform(t)

array([0, 1])

In [0]:
# separate labels 'X' from features 'y'
X = df.iloc[:, np.r_[2,5,6,7,8,9,10,11,12]] # skip PatientId and AppointmentID
y = df.iloc[:, np.r_[13]]

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y, random_state=1)

In [7]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc2


In [0]:
CATEGORICAL_COLUMNS = ['Gender','Neighbourhood','Scholarship','Hipertension','Diabetes','Alcoholism','Handcap','SMS_received']
NUMERIC_COLUMNS=['Age']

fc=tf.feature_column

feature_columns = []

feature_columns.append(tf.feature_column.numeric_column('Age',dtype=tf.float32))

for feat_col in CATEGORICAL_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feat_col,dtype=tf.int32))
  

In [0]:
NUM_EXAMPLES = 200
def make_input_fn(X, y, n_epochs=None, shuffle=True):
  def input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((dict(X),y))
    if shuffle:
        dataset = dataset.shuffle(NUM_EXAMPLES)
    dataset = dataset.repeat(n_epochs)
    dataset = dataset.batch(NUM_EXAMPLES)
    return dataset
  return input_fn

In [0]:
train_input_fn = make_input_fn(X_train, y_train)
eval_input_fn = make_input_fn(X_test, y_test, shuffle=False, n_epochs=1)

In [15]:
!pip install tensorboardcolab


In [17]:

tbc=TensorBoardColab

NameError: ignored

In [15]:
%%time
linear_est = tf.estimator.LinearClassifier(feature_columns)
linear_est.train(train_input_fn, max_steps=300)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpk_v_3c97', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.


In [14]:
%%time
result = linear_est.evaluate(eval_input_fn)
#clear_output()
print(pd.Series(result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-04-12T20:36:31Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpy69qfb4p/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.88071s
INFO:tensorflow:Finished evaluation at 2020-04-12-20:36:32
INFO:tensorflow:Saving dict for global step 100: accuracy = 0.7980639, accuracy_baseline = 0.7980639, auc = 0.55435884, auc_precision_recall = 0.2283003, average_loss = 0.57460165, global_step = 100, label/mean = 0.20193613, loss = 0.57407975, precision = 0.5, prediction/mean = 0.10898576, recall = 0.00022401434
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 100: /tmp/tmpy69qfb4p/model.ckpt-100
accuracy                  0.798064
accuracy_baseline         0.798064
auc                       0.554359
auc_precision_recall      0.228300
average_loss    